In [1]:
import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from src.multimodal import MultimodalRetrieval

/Users/luiss/.pyenv/versions/3.11.3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
multimodal = MultimodalRetrieval()

filepath = os.path.join(kit_dir,"data/sample_docs/invoicesample.pdf")

In [3]:
raw_pdf_elements, output_path = multimodal.extract_pdf(filepath)

2024-08-27 08:47:47,619 [INFO] - pikepdf C++ to Python logger bridge initialized
2024-08-27 08:47:49,708 [INFO] - Reading PDF for file: /Users/luiss/Documents/projects/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample.pdf ...
2024-08-27 08:47:53,697 [INFO] - Loading the Table agent ...
2024-08-27 08:47:53,698 [INFO] - Loading the table structure model ...
2024-08-27 08:47:54,258 [INFO] - Loading pretrained weights from Hugging Face hub (timm/resnet18.a1_in1k)
2024-08-27 08:47:54,485 [INFO] - [timm/resnet18.a1_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
2024-08-27 08:47:54,551 [INFO] - Missing keys (fc.weight, fc.bias) discovered while loading pretrained weights. This is expected if model is being adapted.
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_enco

In [4]:
text_docs, table_docs, image_paths = multimodal.process_raw_elements(raw_pdf_elements, output_path)

In [5]:
retriever = multimodal.create_vectorstore()

/Users/luiss/.pyenv/versions/3.11.3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [6]:
retriever = multimodal.vectorstore_ingest(retriever, text_docs, table_docs, image_paths, summarize_texts=True, summarize_tables=True)

In [7]:
qachain = multimodal.get_retrieval_chain(retriever, image_retrieval_type="summary")

In [8]:
qachain("how many apples they bought")

{'question': 'how many apples they bought',
 'answer': 'According to the provided context, they bought 1 apple.',
 'source_documents': [Document(metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-07-29T09:07:11', 'page_number': 1, 'orig_elements': 'eJzNV11v2zYU/SuEsALbYAq8JMWPvPVpA7ZlBeI+DEFgUCLlEJAlV6K2Zl3/+65kuTVWr00HOMuTzGMeiTznnivq9l0WmrALbdpEn12RrPBClQqAlowpKn1VUOMNo0prxmxpQHuRrUi2C8l5lxxy3mVV1/U+ti6FYR437qEb0+Y+xO19QoQLYZGzwH9En+4RBVUIRPddbNPEu71VOsd5YAvI5d2KLGPOtD6MjRa5OQMsDESy4WFIYTft5FV8G5qbvatC9h7/8CGFKsWu3VSNG4bNvu9KnMZyZQXTOKGOTUgP+zBzX/2SzQtut6Pbzru6zUK7ze5mdEibXedjHcOsGWdcUqYpt2tmr5i+ApjYe2Ru2nFXhn7a7bSIFN5OemTrH19e/0R++/X1NO/40HVMzbzWf3oCzGpdg6FO6YJKNIEaWQNFc3StpQq1got5UvAcUHIAlttJ8uPYqpzPFiiby7PAzPhvnljOJH9iT74nP3d92JG4H8Yd8V3T9WSIiTiUdUWqrh1wrSGNPXE+7uNQxXZLQhNTTl428c3odmTovGvCQLzbx3IcSB36ycZxl5Prsa1OiCuyc9vWkSF4MlRodx+HN2Mg1dgP47AioXeJNPjA8ePdxr51q5lRuyo2cYgDTumIHyMJ6bDu/LSgrl2Pt4m/h/W0xTOFpUDxmjOg2mkMe+0FtUoZWuraYdpVsN5frLAs6JxN2Z2u

In [9]:
qachain("whats the address of the store")

{'question': 'whats the address of the store',
 'answer': 'Sure, I can help you with that! The address of the store is 221 Queen St Melbourne VIC 3000.',
 'source_documents': [Document(page_content='ATTENTION TO\n\nDenny Gunawan\n\n221 Queen St Melbourne VIC 3000\n\n123 Somewhere St, Melbourne VIC 3000 (03) 1234 5678\n\n$39.60\n\nInvoice Number: #20130304', metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-05-08T08:57:57', 'page_number': 1, 'orig_elements': 'eJy9ltuO2zYQhl9FUHvRAqbE82HviqYo9iKbFOv0ZrEQKGrkZVaWBIleZxP03UtJTpCDEyAGbMAwML9mJHI+zgzvPqTQwBbaUPgqvUpSbRyRWilEMWDEa6mQkVQip4krZQlGaJ2uknQLwVY22BjzIXVdN1S+tQHG2W7sc7cLxQP4zUOICmXMxJiDvPdVeIgqkYJFte98G6a4uztjaCZWCeE84/er5KNNiczEZBOBVUaPCEtEVNLxeQywnXby2r+D5ra3DtL/4oMKArjgu7ZwjR3Hoh+6MrrhjEomZHSofQPhuYc59vXLdF5wu9nZzbyruxTaTXo/q2Motl3law9zziimHGGBsF5jfSVU/E3RfYws2t22hGHa7bSIAO+mfKR/rNd/3ayvX90k61eT68fvrn1o5uV+jUU5kFJpgSopKOLARMRC4h9lBAzllTsjFqJEpmLWBc7InPXFlkJkbLI14ROmr+3F/zQomsnvMPHbKa29ndea5m9GG

In [10]:
qachain("what is the main color in the business logo")

{'question': 'what is the main color in the business logo',
 'answer': 'The main color in the business logo is gold and yellow.',
 'source_documents': [Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, which represents the company's focus on fresh, high-quality produce. The logo is displayed on a white background, making it stand out.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}),
  Document(page_content='ATTENTION TO\n\nDenny Gunawan\n\n221 Queen St Melbourne VIC 3000\n\n123 Somewhere St, Melbourne VIC 3000 (03) 1234 5678\n\n$39.60\n\nInvoice Number: #20130304', metadata={'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2024-05-08T08:57:57', 'page_

In [11]:
qachain("what is written in the logo")

{'question': 'what is written in the logo',
 'answer': 'The logo features the name "Sunny Farm" in a gold and yellow color scheme, with a sun in the center, surrounded by a green field.',
 'source_documents': [Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, which represents the company's focus on fresh, high-quality produce. The logo is displayed on a white background, making it stand out.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}),
  Document(page_content='THANK YOU\n\n* Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam sodales dapibus fermentum. Nunc adipiscing, magna sed scelerisque cursus, erat lectus dapibus urna, sed facilisis leo dui et ipsum.\

In [12]:
qachain = multimodal.get_retrieval_chain(retriever, image_retrieval_type="raw")

In [13]:
qachain("how many apples they bought")

{'question': 'how many apples they bought',
 'answer': 'Sure, I can help you with that! Based on the provided context, it appears that the customer bought 1 apple, which is reflected in the third column of the table. Therefore, the answer to your question is 1.',
 'source_documents': [Document(page_content='<table><tr><td>Apple</td><td>$5.00</td><td>1</td><td>$5.00</td></tr><tr><td>Orange</td><td>$1.99</td><td>2</td><td>$3.98</td></tr><tr><td>Watermelon</td><td>$1.69</td><td>3</td><td>$5.07</td></tr><tr><td>Mango</td><td>$9.56</td><td>2</td><td>$19.12</td></tr><tr><td>Peach</td><td>$2.99</td><td>1</td><td>$2.99</td></tr></table>', metadata={'last_modified': '2024-05-08T08:57:57', 'text_as_html': '<table><tr><td>Apple</td><td>$5.00</td><td>1</td><td>$5.00</td></tr><tr><td>Orange</td><td>$1.99</td><td>2</td><td>$3.98</td></tr><tr><td>Watermelon</td><td>$1.69</td><td>3</td><td>$5.07</td></tr><tr><td>Mango</td><td>$9.56</td><td>2</td><td>$19.12</td></tr><tr><td>Peach</td><td>$2.99</td><td>

In [14]:
qachain("what is the main color in the business logo")

{'question': 'what is the main color in the business logo',
 'answer': 'Sure, I can help you with that! Based on the provided context, the main color in the business logo is yellow.',
 'source_documents': [Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, which represents the company's focus on fresh, high-quality produce. The logo is displayed on a white background, making it stand out.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}),
  Document(page_content='ATTENTION TO\n\nDenny Gunawan\n\n221 Queen St Melbourne VIC 3000\n\n123 Somewhere St, Melbourne VIC 3000 (03) 1234 5678\n\n$39.60\n\nInvoice Number: #20130304', metadata={'filetype': 'application/pdf', 'languages': [

In [15]:
qachain("what is written in the logo")

{'question': 'what is written in the logo',
 'answer': 'Sure, I can help you with that! Based on the context you provided, the logo says "Sunny Farm Victoria."',
 'source_documents': [Document(page_content="The image features a logo for Sunny Farm, a company that specializes in fresh produce. The logo is a gold and yellow color scheme, with a sun in the center. The sun is surrounded by a green field, which represents the company's focus on fresh, high-quality produce. The logo is displayed on a white background, making it stand out.", metadata={'type': 'image', 'file_directory': '/Users/jorgep/Documents/ask_public_own/ai-starter-kit/multimodal_knowledge_retriever/data/sample_docs/invoicesample', 'filename': 'figure-1-1.jpg'}),
  Document(page_content='THANK YOU\n\n* Lorem ipsum dolor sit amet, consectetur adipiscing elit. Aliquam sodales dapibus fermentum. Nunc adipiscing, magna sed scelerisque cursus, erat lectus dapibus urna, sed facilisis leo dui et ipsum.\n\nSubtotal | Total\n\nSub